In [2]:
from bs4 import BeautifulSoup
import requests
import re
import pymongo  
from pymongo import MongoClient
import pyodbc
import time
from datetime import datetime
import csv
import MongoConnect
import pandas as pd
from selenium import webdriver

headers = {
    'User-Agent':"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/22.0.1207.1 Safari/537.1"
}  



# 先讀csv，讀取歌手的url

In [3]:
Artist_csv=pd.read_csv('../data/Ultimate_Guitar_Artist_cleaned.csv',encoding="ISO-8859-1")

In [15]:
#約200位歌手
Artist_csv.head()

,artist_name,artist_url
0,*NSync,https://www.ultimate-guitar.com/artist/nsync_787
1,10cc,https://www.ultimate-guitar.com/artist/10cc_2049
2,a-ha,https://www.ultimate-guitar.com/artist/a-ha_8667
3,ABBA,https://www.ultimate-guitar.com/artist/abba_1998
4,Ace of Base,https://www.ultimate-guitar.com/artist/ace_of_...


In [53]:
###一個artist_url可以前往一位歌手的吉他譜列表，假設一位歌手的吉他譜總共14頁(14頁算多了)，以判斷有無下一頁做為停止條件，否則就爬到第14頁
###這邊把target_url存入mongo，就可以得到每個歌手有幾頁的吉他譜

db=MongoConnect.mongoconnect()
one_document=db.Ultimate_Guitar_Html_byArtists
for artist_url in Artist_csv.artist_url:
    
    for page in range(1,14):
        try:
            

            
            target_url=artist_url+'?filter=chords&page='+str(page)
            
            all_html=requests.get(target_url,headers = headers)    
            soup1=BeautifulSoup(all_html.text,'lxml')
            
            
              
            bag={
                'page':page,
                'url':target_url,
                'error':0

            }

            print('Page:'+str(page)+'URL:'+target_url)
        except Exception as e:
            print(e)
            bag={
                'page':page,
                'url':target_url,
                'error':1
            }
            #抓取失敗
            print(e)
            print("Error find!Connect again...")
            time.sleep(10)

        finally:
           ###連接MONGO

            #####此層for迴圈終止條件:
            #####這個page是404的話，break掉迴圈跳到下一個artist_url
            if soup1.find('div',class_='b-error-page')  is not None:break
                
            print(page)
            current_insert=one_document.find({'url':target_url}).count()
           
            if(current_insert==0):one_document.insert_one(bag)
            if(current_insert>0):
                print("已存在此頁面")
            
            time.sleep(10)    




Page:1URL:https://www.ultimate-guitar.com/artist/nsync_787?filter=chords&page=1
1
Page:2URL:https://www.ultimate-guitar.com/artist/nsync_787?filter=chords&page=2
Page:1URL:https://www.ultimate-guitar.com/artist/10cc_2049?filter=chords&page=1
1
Page:2URL:https://www.ultimate-guitar.com/artist/10cc_2049?filter=chords&page=2
Page:1URL:https://www.ultimate-guitar.com/artist/a-ha_8667?filter=chords&page=1
1
Page:2URL:https://www.ultimate-guitar.com/artist/a-ha_8667?filter=chords&page=2
2
Page:3URL:https://www.ultimate-guitar.com/artist/a-ha_8667?filter=chords&page=3
Page:1URL:https://www.ultimate-guitar.com/artist/abba_1998?filter=chords&page=1
1
Page:2URL:https://www.ultimate-guitar.com/artist/abba_1998?filter=chords&page=2
2
Page:3URL:https://www.ultimate-guitar.com/artist/abba_1998?filter=chords&page=3
Page:1URL:https://www.ultimate-guitar.com/artist/ace_of_base_2870?filter=chords&page=1
1
Page:2URL:https://www.ultimate-guitar.com/artist/ace_of_base_2870?filter=chords&page=2
Page:1URL:ht

Page:1URL:https://www.ultimate-guitar.com/artist/britney_spears_11380?filter=chords&page=1
1
Page:2URL:https://www.ultimate-guitar.com/artist/britney_spears_11380?filter=chords&page=2
2
Page:3URL:https://www.ultimate-guitar.com/artist/britney_spears_11380?filter=chords&page=3
Page:1URL:https://www.ultimate-guitar.com/artist/bruno_mars_28280?filter=chords&page=1
1
Page:2URL:https://www.ultimate-guitar.com/artist/bruno_mars_28280?filter=chords&page=2
2
Page:3URL:https://www.ultimate-guitar.com/artist/bruno_mars_28280?filter=chords&page=3
3
Page:4URL:https://www.ultimate-guitar.com/artist/bruno_mars_28280?filter=chords&page=4
Page:1URL:https://www.ultimate-guitar.com/artist/buggles_1820?filter=chords&page=1
1
Page:2URL:https://www.ultimate-guitar.com/artist/buggles_1820?filter=chords&page=2
Page:1URL:https://www.ultimate-guitar.com/artist/carly_rae_jepsen_24312?filter=chords&page=1
1
Page:2URL:https://www.ultimate-guitar.com/artist/carly_rae_jepsen_24312?filter=chords&page=2
2
Page:3URL:h

Page:2URL:https://www.ultimate-guitar.com/artist/dusty_springfield_11385?filter=chords&page=2
Page:1URL:https://www.ultimate-guitar.com/artist/ed_sheeran_30232?filter=chords&page=1
1
Page:2URL:https://www.ultimate-guitar.com/artist/ed_sheeran_30232?filter=chords&page=2
2
Page:3URL:https://www.ultimate-guitar.com/artist/ed_sheeran_30232?filter=chords&page=3
3
Page:4URL:https://www.ultimate-guitar.com/artist/ed_sheeran_30232?filter=chords&page=4
4
Page:5URL:https://www.ultimate-guitar.com/artist/ed_sheeran_30232?filter=chords&page=5
5
Page:6URL:https://www.ultimate-guitar.com/artist/ed_sheeran_30232?filter=chords&page=6
6
Page:7URL:https://www.ultimate-guitar.com/artist/ed_sheeran_30232?filter=chords&page=7
Page:1URL:https://www.ultimate-guitar.com/artist/edwin_mccain_10861?filter=chords&page=1
1
Page:2URL:https://www.ultimate-guitar.com/artist/edwin_mccain_10861?filter=chords&page=2
Page:1URL:https://www.ultimate-guitar.com/artist/electric_light_orchestra_1494?filter=chords&page=1
1
Pag

Page:1URL:https://www.ultimate-guitar.com/artist/gwen_stefani_9015?filter=chords&page=1
1
Page:2URL:https://www.ultimate-guitar.com/artist/gwen_stefani_9015?filter=chords&page=2
Page:1URL:https://www.ultimate-guitar.com/artist/hanson_1276?filter=chords&page=1
1
Page:2URL:https://www.ultimate-guitar.com/artist/hanson_1276?filter=chords&page=2
2
Page:3URL:https://www.ultimate-guitar.com/artist/hanson_1276?filter=chords&page=3
Page:1URL:https://www.ultimate-guitar.com/artist/harry_nilsson_11018?filter=chords&page=1
1
Page:2URL:https://www.ultimate-guitar.com/artist/harry_nilsson_11018?filter=chords&page=2
2
Page:3URL:https://www.ultimate-guitar.com/artist/harry_nilsson_11018?filter=chords&page=3
Page:1URL:https://www.ultimate-guitar.com/artist/helene_fischer_37027?filter=chords&page=1
1
Page:2URL:https://www.ultimate-guitar.com/artist/helene_fischer_37027?filter=chords&page=2
Page:1URL:https://www.ultimate-guitar.com/artist/hooverphonic_8672?filter=chords&page=1
1
Page:2URL:https://www.ul

Page:1URL:https://www.ultimate-guitar.com/artist/kiki_dee_32061?filter=chords&page=1
1
Page:2URL:https://www.ultimate-guitar.com/artist/kiki_dee_32061?filter=chords&page=2
Page:1URL:https://www.ultimate-guitar.com/artist/kim_carnes_9979?filter=chords&page=1
1
Page:2URL:https://www.ultimate-guitar.com/artist/kim_carnes_9979?filter=chords&page=2
Page:1URL:https://www.ultimate-guitar.com/artist/king_krule_35813?filter=chords&page=1
1
Page:2URL:https://www.ultimate-guitar.com/artist/king_krule_35813?filter=chords&page=2
Page:1URL:https://www.ultimate-guitar.com/artist/lady_antebellum_21226?filter=chords&page=1
1
Page:2URL:https://www.ultimate-guitar.com/artist/lady_antebellum_21226?filter=chords&page=2
2
Page:3URL:https://www.ultimate-guitar.com/artist/lady_antebellum_21226?filter=chords&page=3
Page:1URL:https://www.ultimate-guitar.com/artist/lady_gaga_21818?filter=chords&page=1
1
Page:2URL:https://www.ultimate-guitar.com/artist/lady_gaga_21818?filter=chords&page=2
2
Page:3URL:https://www.

Page:2URL:https://www.ultimate-guitar.com/artist/natasha_bedingfield_9800?filter=chords&page=2
Page:1URL:https://www.ultimate-guitar.com/artist/nathan_sykes_59869?filter=chords&page=1
1
Page:2URL:https://www.ultimate-guitar.com/artist/nathan_sykes_59869?filter=chords&page=2
Page:1URL:https://www.ultimate-guitar.com/artist/naughty_boy_40620?filter=chords&page=1
1
Page:2URL:https://www.ultimate-guitar.com/artist/naughty_boy_40620?filter=chords&page=2
Page:1URL:https://www.ultimate-guitar.com/artist/neil_diamond_10178?filter=chords&page=1
1
Page:2URL:https://www.ultimate-guitar.com/artist/neil_diamond_10178?filter=chords&page=2
2
Page:3URL:https://www.ultimate-guitar.com/artist/neil_diamond_10178?filter=chords&page=3
Page:1URL:https://www.ultimate-guitar.com/artist/nelly_furtado_10323?filter=chords&page=1
1
Page:2URL:https://www.ultimate-guitar.com/artist/nelly_furtado_10323?filter=chords&page=2
Page:1URL:https://www.ultimate-guitar.com/artist/nick_jonas_29714?filter=chords&page=1
1
Page:

Page:2URL:https://www.ultimate-guitar.com/artist/rupert_holmes_11962?filter=chords&page=2
Page:1URL:https://www.ultimate-guitar.com/artist/sade_290?filter=chords&page=1
1
Page:2URL:https://www.ultimate-guitar.com/artist/sade_290?filter=chords&page=2
Page:1URL:https://www.ultimate-guitar.com/artist/sara_bareilles_16977?filter=chords&page=1
1
Page:2URL:https://www.ultimate-guitar.com/artist/sara_bareilles_16977?filter=chords&page=2
2
Page:3URL:https://www.ultimate-guitar.com/artist/sara_bareilles_16977?filter=chords&page=3
Page:1URL:https://www.ultimate-guitar.com/artist/sarah_mclachlan_10892?filter=chords&page=1
1
Page:2URL:https://www.ultimate-guitar.com/artist/sarah_mclachlan_10892?filter=chords&page=2
2
Page:3URL:https://www.ultimate-guitar.com/artist/sarah_mclachlan_10892?filter=chords&page=3
Page:1URL:https://www.ultimate-guitar.com/artist/savage_garden_507?filter=chords&page=1
1
Page:2URL:https://www.ultimate-guitar.com/artist/savage_garden_507?filter=chords&page=2
Page:1URL:https

Page:10URL:https://www.ultimate-guitar.com/artist/the_beatles_1916?filter=chords&page=10
10
Page:11URL:https://www.ultimate-guitar.com/artist/the_beatles_1916?filter=chords&page=11
11
Page:12URL:https://www.ultimate-guitar.com/artist/the_beatles_1916?filter=chords&page=12
12
Page:13URL:https://www.ultimate-guitar.com/artist/the_beatles_1916?filter=chords&page=13
13
Page:1URL:https://www.ultimate-guitar.com/artist/the_black_eyed_peas_3119?filter=chords&page=1
1
Page:2URL:https://www.ultimate-guitar.com/artist/the_black_eyed_peas_3119?filter=chords&page=2
Page:1URL:https://www.ultimate-guitar.com/artist/the_corrs_1778?filter=chords&page=1
1
Page:2URL:https://www.ultimate-guitar.com/artist/the_corrs_1778?filter=chords&page=2
2
Page:3URL:https://www.ultimate-guitar.com/artist/the_corrs_1778?filter=chords&page=3
Page:1URL:https://www.ultimate-guitar.com/artist/the_drifters_1627?filter=chords&page=1
1
Page:2URL:https://www.ultimate-guitar.com/artist/the_drifters_1627?filter=chords&page=2
Pag

Page:1URL:https://www.ultimate-guitar.com/artist/years_years_46438?filter=chords&page=1
1
Page:2URL:https://www.ultimate-guitar.com/artist/years_years_46438?filter=chords&page=2
2
Page:3URL:https://www.ultimate-guitar.com/artist/years_years_46438?filter=chords&page=3
3
Page:4URL:https://www.ultimate-guitar.com/artist/years_years_46438?filter=chords&page=4
4
Page:5URL:https://www.ultimate-guitar.com/artist/years_years_46438?filter=chords&page=5
5
Page:6URL:https://www.ultimate-guitar.com/artist/years_years_46438?filter=chords&page=6
6
Page:7URL:https://www.ultimate-guitar.com/artist/years_years_46438?filter=chords&page=7
7
Page:8URL:https://www.ultimate-guitar.com/artist/years_years_46438?filter=chords&page=8
8
Page:9URL:https://www.ultimate-guitar.com/artist/years_years_46438?filter=chords&page=9
9
Page:10URL:https://www.ultimate-guitar.com/artist/years_years_46438?filter=chords&page=10
10
Page:11URL:https://www.ultimate-guitar.com/artist/years_years_46438?filter=chords&page=11
11
Page

以下把歌手列表html從mongo撈出來再把每個吉他譜爬出來(一頁html有好多張吉他譜)  <br>
例如一頁有50個吉他譜連結，前往連結頁面把網頁html爬下來存在Mongo，之後再拿出來解析

In [3]:
db=MongoConnect.mongoconnect()
one_document=db.Ultimate_Guitar_Html_byArtists
list=one_document.find({'error':0})
data=[]
load=0
for i in list:
    load=load+1
    if(load % 100==0):print(load)
    data.append(i)


100
200
300
400
500


In [1]:
for i in data:
    print(i['url'])

In [5]:
#目前爬到:data_row=167
data_row=0
driver = webdriver.Chrome()     # 打開Chrome 瀏覽器，webdriver.Chrome('下載的Chrome driver路徑.exe')
for i in data:
    data_row=data_row+1
    if data_row<=167:continue
    print('data_row:',data_row)
    
    #for each 網頁(一頁幾十個吉他譜要抓)
    page=i['page']
    url=i['url']
    print('Page:',page)
    print('url:',url)
    try:
        driver.get(url)#請求網址
        page_html = driver.page_source
        soup1=BeautifulSoup(page_html,'lxml')
        each_tab_href=soup1.find_all('a',class_='_30ssZ _2gHWP _2-NA5')
        
        crawl_num=0
        for tab in each_tab_href:
            ##each_tab_href:一個頁面上有多個譜的超連結
            try:
                crawl_num=crawl_num+1
                
                if(crawl_num % 5==0):time.sleep(10) #每抓5篇就多休息15秒
                
                ##for each 譜超連結:

                tab_href=tab['href']
                tab_title=tab.text
                driver.get(tab_href)#請求譜的網址
                
                try:#先按下網頁上View all tabs按鈕之後再拿html
                    driver.find_element_by_class_name('ChIQl').find_element_by_class_name('gGpLU').click()
                except Exception as e:
                    #取得網頁html
                    print('此tab頁面沒有顯示所有tab的按鈕')
                finally:
                    tab_html = driver.page_source       #tab_html=driver.page_source

                bag={

                    'tab_href':tab_href,
                    'web_html':tab_html,
                    'tab_title':tab_title,
                    'error':0

                }
            except Exception as e:
                print('爬這張譜時有問題',tab_url)

                bag={
                    'url':tab_url,
                    'web_html':tab_html,
                    'error':1

                }
            finally:###連接MONGO



                #####此層for迴圈終止條件:
                #####這個page是404的話，break掉迴圈跳到下一張譜
                if soup1.find('div',class_='b-error-page')  is not None:
                    print('錯誤頁面')
                    time.sleep(5)
                    break
                
                db=MongoConnect.mongoconnect()
                one_document=db.Ultimate_Guitar_EachTAB_Html_byArtists
                one_document.insert_one(bag)
                print('insert success:',tab_href)
                time.sleep(10)   

    except Exception as e:
        print('打開譜的列表頁面時發生錯誤:',e)
        time.sleep(10)


data_row: 168
Page: 1
url: https://www.ultimate-guitar.com/artist/gerry_the_pacemakers_5344?filter=chords&page=1
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/gerry_the_pacemakers/a_shot_of_rhythm_and_blues_chords_1974061
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/gerry_the_pacemakers/a_shot_of_rhythm_and_blues_chords_1974001
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/gerry_the_pacemakers/away_from_you_chords_909808
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/gerry_the_pacemakers/away_from_you_chords_914924
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/gerry_the_pacemakers/baby_youre_so_good_to_me_chords_1715229
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/gerry_the_pacemakers/come_back_to_me_chords_1714413
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/gerry_the_pacemakers/dont_let_the_sun_catch_yo

此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/glen_campbell/a_daisy_a_day_chords_1787330
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/glen_campbell/a_few_good_men_chords_1787332
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/glen_campbell/a_lady_like_you_chords_1787529
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/glen_campbell/a_place_in_the_sun_chords_1785646
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/glen_campbell/a_satisfied_mind_chords_1787533
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/glen_campbell/a_song_for_you_chords_1787537
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/glen_campbell/a_thing_called_love_chords_1788191
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/glen_campbell/a_thousand_lifetimes_chords_1802042
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimat

此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/glen_campbell/call_it_even_chords_1788842
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/glen_campbell/can_you_fool_chords_1789366
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/glen_campbell/careless_weed_chords_1789369
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/glen_campbell/carolina_in_my_mind_chords_1821362
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/glen_campbell/catch_the_wind_chords_1789371
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/glen_campbell/cheatin_is_chords_1789381
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/glen_campbell/christian_no_chords_1789384
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/glen_campbell/christmas_day_chords_1813796
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/glen_ca

insert success: https://tabs.ultimate-guitar.com/tab/glen_campbell/gentle_on_my_mind_chords_64336
insert success: https://tabs.ultimate-guitar.com/tab/glen_campbell/gentle_on_my_mind_chords_578888
insert success: https://tabs.ultimate-guitar.com/tab/glen_campbell/gentle_on_my_mind_chords_1720947
insert success: https://tabs.ultimate-guitar.com/tab/glen_campbell/gentle_on_my_mind_chords_1824949
insert success: https://tabs.ultimate-guitar.com/tab/glen_campbell/gentle_on_my_mind_chords_1831520
insert success: https://tabs.ultimate-guitar.com/tab/glen_campbell/gentle_on_my_mind_chords_2007275
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/glen_campbell/ghost_on_the_canvas_chords_1789815
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/glen_campbell/give_back_that_old_familiar_feeling_chords_1790059
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/glen_campbell/god_only_knows_chords_1804416
此tab頁面沒有顯示所有tab的按鈕
insert suc

此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/glen_campbell/i_want_to_be_with_you_always_chords_1808934
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/glen_campbell/i_was_just_thinking_about_you_chords_1817499
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/glen_campbell/i_was_too_busy_lovin_you_chords_1797077
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/glen_campbell/i_will_arise_chords_1797078
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/glen_campbell/i_will_be_here_chords_1816503
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/glen_campbell/i_will_never_pass_this_way_again_chords_1797079
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/glen_campbell/id_build_a_bridge_chords_1800629
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/glen_campbell/if_chords_1841754
此tab頁面沒有顯示所有tab的按鈕
insert 

...

此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/whitney_houston/i_will_always_love_you_chords_416037
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/whitney_houston/i_will_always_love_you_chords_878373
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/whitney_houston/i_will_always_love_you_chords_1428255
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/whitney_houston/i_will_always_love_you_chords_2136795
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/whitney_houston/if_you_say_my_eyes_are_beautiful_chords_2078531
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/whitney_houston/im_your_baby_tonight_chords_1718243
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/whitney_houston/im_your_baby_tonight_chords_1911639
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/whitney_houston/its_not_right_but_its_ok_c

此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/wiz_khalifa/purple_flowers_chords_1994177
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/wiz_khalifa/roll_up_chords_1042808
insert success: https://tabs.ultimate-guitar.com/tab/wiz_khalifa/see_you_again_chords_1720298
insert success: https://tabs.ultimate-guitar.com/tab/wiz_khalifa/see_you_again_chords_1727693
insert success: https://tabs.ultimate-guitar.com/tab/wiz_khalifa/see_you_again_chords_1730241
insert success: https://tabs.ultimate-guitar.com/tab/wiz_khalifa/see_you_again_chords_1729821
insert success: https://tabs.ultimate-guitar.com/tab/wiz_khalifa/see_you_again_chords_1733222
insert success: https://tabs.ultimate-guitar.com/tab/wiz_khalifa/see_you_again_chords_1736282
insert success: https://tabs.ultimate-guitar.com/tab/wiz_khalifa/see_you_again_chords_1738318
insert success: https://tabs.ultimate-guitar.com/tab/wiz_khalifa/see_you_again_chords_1755087
insert success: https://tab

data_row: 483
Page: 12
url: https://www.ultimate-guitar.com/artist/woodkid_33586?filter=chords&page=12
data_row: 484
Page: 13
url: https://www.ultimate-guitar.com/artist/woodkid_33586?filter=chords&page=13
data_row: 485
Page: 1
url: https://www.ultimate-guitar.com/artist/years_years_46438?filter=chords&page=1
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/jax_jones/play_chords_2592222
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/years_years/1977_chords_1797678
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/years_years/all_for_you_chords_2421097
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/years_years/come_alive_chords_2518869
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/years_years/desire_chords_1729551
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/years_years/dont_panic_chords_2418115
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tab

此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/yui/crossroad_chords_759008
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/yui/daydreamer_chords_663767
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/yui/driving_happy_life_chords_1252064
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/yui/es_car_chords_1077314
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/yui/feel_my_soul_chords_400332
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/yui/feel_my_soul_chords_449394
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/yui/fight_chords_1194109
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/yui/free_bird_chords_812815
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/yui/get_back_home_chords_1152304
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/yui/gloria_

此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/yui/tokyo_chords_1455719
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/yui/tomorrows_way_chords_475853
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/yui/tomorrows_way_chords_488766
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/yui/understand_chords_534322
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/yui/why_chords_581867
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/yui/why_me_chords_667904
data_row: 499
Page: 2
url: https://www.ultimate-guitar.com/artist/yui_15012?filter=chords&page=2
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/yui/winding_road_chords_2531331
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/yui/you_chords_1150953
此tab頁面沒有顯示所有tab的按鈕
insert success: https://tabs.ultimate-guitar.com/tab/yui/your_heaven_chords_1083501
此tab